In [1]:
# General libraries
import pandas as pd
import numpy as np

# Scikit Learn libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

# Scipy libraries
from scipy import stats
import joblib
import warnings
warnings.filterwarnings('ignore')

# Utils functions
from utils.utils import kfold, five_two, read_datasets

# Load Dataset

In [2]:
x_train, x_test, y_train, y_test = read_datasets(
    'x_train.csv',
    'x_test.csv',
    'y_train.csv',
    'y_test.csv'
)

# Support Vector Regression (SVR)

In [3]:
# The cross validation scheme to be used for train and test
folds = kfold()

In [4]:
# Create a SVR model with cross validation and default parameters
svr = SVR(kernel='rbf', C=1.0, gamma='auto', epsilon=0.1, degree=3)

## Grid Search

In [5]:
# Specify range of hyperparameters to tune
hyper_params = {
    'kernel': ('linear', 'rbf','poly', 'sigmoid'),
    'C':[1, 1.5, 5, 10, 100],
    'gamma': [1e-7, 1e-4, 'auto', 'scale'],
    'epsilon':[0.1,0.2,0.3,0.4,0.5],
    'degree': [1,2,3,4]
    }


# Call GridSearchCV()
model_cv = GridSearchCV(
    estimator = SVR(),
    param_grid = hyper_params,
    scoring= 'r2',
    cv = folds,
    verbose = 1,
    return_train_score=True,
    n_jobs = -1,
    refit = True
    )


# Fit the model
best_model = model_cv.fit(x_train, np.ravel(y_train)) 

print(model_cv.best_params_)

Fitting 10 folds for each of 1600 candidates, totalling 16000 fits
{'C': 10, 'degree': 2, 'epsilon': 0.1, 'gamma': 'scale', 'kernel': 'poly'}


In [6]:
# Create new model with best_params_ from grid search

svr_best = best_model.best_estimator_

In [7]:
# Get the results for each split

def get_best_model_cv_split_results(best_model, n_splits=10, set_type='train'):
    results = []
    best_index = best_model.best_index_
    for i in range(0, n_splits):
        current_split = 'split{}_{}_score'.format(i, set_type)
        split_result = best_model.cv_results_[current_split][best_index]
        results.append(split_result)

    return results

print("Train Results: {}".format(get_best_model_cv_split_results(best_model, 10, 'train')))
print("Test Results: {}".format(get_best_model_cv_split_results(best_model, 10, 'test')))

Train Results: [0.2449845184409153, 0.24671253738174037, 0.22688614479296676, 0.2396351851711167, 0.25832170284743305, 0.2981504783462676, 0.25189229386532197, 0.25460737558499014, 0.2480827991202349, 0.23813494819473513]
Test Results: [0.2625022850749761, 0.10467198420613932, 0.39127596504957196, 0.2715635146099973, 0.22672019392391485, -0.31933591059322075, 0.20390555701599544, 0.04469261682705328, 0.1256176235187545, 0.29059931309576503]


In [8]:
#Get the mean for the train and test

train_mean = sum(get_best_model_cv_split_results(best_model, 10, 'train'))/10
test_mean = sum(get_best_model_cv_split_results(best_model, 10, 'test'))/10

print("Train mean: {}".format(train_mean))
print("Test mean: {}".format(test_mean))

Train mean: 0.25074079837457225
Test mean: 0.1602213142728947


In [9]:
r2 = r2_score(y_test, svr_best.predict(x_test))
print("The r2 score on test set: {:.4f}".format(r2))

The r2 score on test set: 0.1072


## Statistical hypothesis testing

Validate if the grid model is better than the base model

Null hyphotesis and Alternative hyphotesis
* Ho = Best params R2 and Adj R2 <= base model R2 and Adj R2
* Ha = Best params R2 and Adj R2 > base model R2 and Adj R2

Errors:
* Type I Error: false positive, reject the Ho but it is true
* Type II Error: false negative, do not reject the Ho but its false

In [10]:
five_two(
    reg1=svr,
    reg2=svr_best,
    X=x_train,
    y=y_train
)

Fold  1 score difference = -0.127370
Fold  2 score difference = -0.238147
Fold  1 score difference = -0.055002
Fold  2 score difference = -0.232865
Fold  1 score difference = -0.361331
Fold  2 score difference = -0.116298
Fold  1 score difference = -0.170121
Fold  2 score difference = -0.166749
Fold  1 score difference = -0.245651
Fold  2 score difference = -0.150806
Regression 1 mean score and stdev : -0.030837 + 0.067260
Regression 2 mean score and stdev : 0.155597 + 0.047330
Score difference mean + stdev : -0.186434 + 0.081566
t_value for the current test is -1.198431


In [11]:
five_two(
    reg1=svr,
    reg2=svr_best,
    X=x_train,
    y=y_train,
    metric='adj_r2'
)

Fold  1 score difference = -0.169826
Fold  2 score difference = -0.317529
Fold  1 score difference = -0.073336
Fold  2 score difference = -0.310487
Fold  1 score difference = -0.481775
Fold  2 score difference = -0.155063
Fold  1 score difference = -0.226827
Fold  2 score difference = -0.222332
Fold  1 score difference = -0.327535
Fold  2 score difference = -0.201075
Regression 1 mean score and stdev : -0.374449 + 0.089680
Regression 2 mean score and stdev : -0.125871 + 0.063107
Score difference mean + stdev : -0.248579 + 0.108755
t_value for the current test is -1.198431


In [12]:
from mlxtend.evaluate import paired_ttest_5x2cv

t, p = paired_ttest_5x2cv(estimator1=svr,
                          estimator2=svr_best,
                          X=x_train, y=y_train,
                          scoring='r2',
                          random_seed=42)

print('t statistic: %.3f' % t)
print('p value: %.3f' % p)

t statistic: -2.662
p value: 0.045


# Saving trained model

In [13]:
filename = '../models/svr_model.joblib'
joblib.dump(svr_best, filename)

['../models/svr_model.joblib']

# Conclusions

What was done:
* Create a SVR model with default parameters and another with grid seach + cross validation;
* Compare the test scores (r2 and adj r2) from before the grid search and after using T test using 5 x 2-fold cross validation (5 cv of 2 folds);
* On grid search: C=1000 did not show better results so it was removed
* It seems that the grid model is better than the basemodel with a 80% confidence level